# Feature Selection

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn import preprocessing # LabelEncoder
from sklearn.preprocessing import MinMaxScaler # Escala los datos
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
import scikitplot as skplt 
from string import ascii_uppercase 
# import seaborn as sns
import qgrid
import time
#from sklearn.externals import joblib # Para guardar el modelo
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs

from scipy import stats #Para la moda

from sklearn.model_selection import ShuffleSplit

In [ ]:
def loadCSV(pathSamples, pathMatrix):
    df_loaded = pd.read_table(pathMatrix, sep=',')
    data = pd.read_table(pathSamples, sep=',')
    clin_trial_values = df_loaded.values
    
    Y = data['Eligible']
    Y = Y.astype(int)
    X = clin_trial_values[:, :]

In [ ]:
def select_features(modelo, n_features, fwd, fltg):

    sfs = SFS(modelo, 
           k_features=n_features,
           forward=fwd,
           floating=fltg,
           verbose=1,
           scoring='accuracy',
           cv=0,
           n_jobs=-1)
    
    return sfs

In [ ]:
def selection(features, model):
    
    fwd = True
    fltg = True
        
    #Para calcular el costo computacional
    tiempo_i = time.time()

    accuracy_list = np.zeros([4])
    precision_list = np.zeros([4,2])
    recall_list = np.zeros([4,2])
    f_list = np.zeros([4,2])
    
    feature_list = [] #Para guardar los indices de las características que eligió en cada iteración
    XN_list = [] #Para guardar los diferentes X
    sf_list = []
    
    #Implemetamos la metodología de validación 

    Errores = np.ones(4)
    j = 0
    
    for j in range(4):

        Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.25) # Modificar metodología de validación
        scaler = MinMaxScaler()#Escala entre 0 y 1
        Xtrain = scaler.fit_transform(Xtrain)
        Xtest = scaler.transform(Xtest)

        sf = select_features(model, features, fwd, fltg)
        sf =  sf.fit(Xtrain, Ytrain)

        X_train_sfs = sf.transform(Xtrain)
        X_test_sfs = sf.transform(Xtest)
        XN = sf.transform(X)

        model.fit(X_train_sfs, Ytrain)
        pred = model.predict(X_test_sfs)
        
        #code for calculating accuracy 
        _accuracy_ = accuracy_score(Ytest, pred, normalize=True)
        accuracy_list[j] = _accuracy_

        #code for calculating recall 
        _recalls_ = recall_score(Ytest, pred, average=None)
        recall_list[j] = _recalls_

        #code for calculating precision 
        _precisions_ = precision_score(Ytest, pred, average=None)
        precision_list[j] = _precisions_
        
        _f_score_ = f1_score(Ytest, pred, average=None)
        f_list[j] = _f_score_

        Errores[j] = classification_error(pred, Ytest)
        
        feature_list.extend(sf.k_feature_idx_)
        
        XN_list.append(XN)
        sf_list.append(sf)
    
    
    index = np.where(accuracy_list == np.amax(accuracy_list)) #Busca cual es el resultado más grande en la lista de accuracy(eficiencia)   
    idx = index[0][0] #pasa el índice de array a entero   
    XN = XN_list[idx] #Selecciona el mejor X 
    sf = sf_list[idx] #índices de las mejores características
    
    mode, count = stats.mode(feature_list) #primer retorno es un vector con las modas, el segundo son las repeticiones
    
    return mode, count, XN, sf, str(np.mean(accuracy_list)), str(np.std(accuracy_list)), str(np.mean(recall_list)), str(np.std(recall_list)), str(np.mean(precision_list)), str(np.std(precision_list)),  str(np.mean(f_list)), str(np.std(f_list)), str(np.mean(Errores)), str(np.std(Errores)), str(time.time()-tiempo_i)

# LSA 10k_1Col_NoCarEsp

In [ ]:
loadCSV("../../Dataset/10k_1Col_NoCarEsp_LSA.csv", "../../Tables/docsTopicsLSA1200.csv") #Cargar SCV

In [ ]:
pd.options.mode.chained_assignment = None

randn = np.random.randn
df_types = pd.DataFrame({'# de características seleccionadas' : pd.Series([200,300])})
df_types["Eficiencia"] = ""
df_types["Int_Eficiencia"] = ""
df_types["Sensibilidad"] = ""
df_types["Int_Sensibilidad"] = ""
df_types["Precision"] = ""
df_types["Int_Precision"] = ""
df_types["F-Score"] = ""
df_types["Int_F-Score"] = ""
df_types["Error_Prueba"] = ""
df_types["Int_error"] = ""
df_types["Tiempo de ejecución"] = ""
df_types.set_index(['# de características seleccionadas'], inplace=True)

for k in df_types.index:
    mode, count, XN, sf, Acc, IntAcc, Sen, IntSen, Pre, IntPre, f, IntF, error, stdError, tiempo = selection(k)
    print(k)
    print(sf.k_feature_idx_)
    df_types["Eficiencia"][k] = Acc
    df_types["Int_Eficiencia"][k] = IntAcc
    df_types["Sensibilidad"][k] = Sen
    df_types["Int_Sensibilidad"][k] = IntSen
    df_types["Precision"][k] = Pre
    df_types["Int_Precision"][k] = IntPre
    df_types["F-Score"][k] = f
    df_types["Int_F-Score"][k] = IntF
    df_types["Error_Prueba"][k] = error
    df_types["Int_error"][k] = stdError
    df_types["Tiempo de ejecución"][k] = tiempo
    
qgrid_widget = qgrid.show_grid(df_types, show_toolbar=False)

In [ ]:
mode, count, XN, sf, Acc, IntAcc, Sen, IntSen, Pre, IntPre, f, IntF, error, stdError, tiempo = selection(100)


## LSA, 10k, sin caracteres especiales, 1500 Tópicos


Latent Semantic Analysis con 10mil muestras. Uniendo las columnas "Diagnoses" e "Interventions". Eliminando todos los caracteres especiales y números de las muestras. 1500 tópicos.

In [ ]:
loadCSV("../../Dataset/10k_1Col_NoCarEsp_LSA.csv", "../../Tables/docsTopicsLSA1200.csv") #Cargar SCV